In [1]:
import pandas as pd
pd.set_option('display.max_columns', 1500)

import warnings
warnings.filterwarnings('ignore')

#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
import category_encoders as ce
from copy import deepcopy
import pickle 

C:\Users\차유경\OneDrive\UTD\2023 Spring\Applied Machine Learning\ml-spring-2023\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\차유경\OneDrive\UTD\2023 Spring\Applied Machine Learning\ml-spring-2023\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\차유경\OneDrive\UTD\2023 Spring\Applied Machine Learning\ml-spring-2023\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\차유경\OneDrive\UTD\2023 Spring\Applied Machine Learning\ml-spring-2023\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import h2o
try:
    h2o.init()
    h2o.cluster().shutdown()
except:
    pass 
h2o.init(nthreads=8, max_mem_size=8)
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.341-b10, mixed mode)
  Starting server from C:\Users\차유경\OneDrive\UTD\2023 Spring\Applied Machine Learning\ml-spring-2023\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: D:\tmpb5_v7l82
  JVM stdout: D:\tmpb5_v7l82\h2o_차유경_started_from_python.out
  JVM stderr: D:\tmpb5_v7l82\h2o_차유경_started_from_python.err
Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,19 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.4
H2O_cluster_version_age:,4 months and 2 days !!!
H2O_cluster_name:,H2O_from_python_李⑥쑀寃�_dsvyyb
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.710 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
from h2o.frame import H2OFrame

In [4]:
model=h2o.load_model('GBM_model_python_1683424268951_7550')

In [9]:
class Data_loader:
    def __init__(self,data_path):
        self.data=pd.read_csv(data_path)
        self.data=data
        self.data=self.data.drop(columns=['index'])
    
        # Convert the strings styled as '$XXXX.XX' to float values
        self.Columns = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
        for col in self.Columns:
            self.data[col] = self.data[col].str.replace('$', '')
            self.data[col] = self.data[col].str.replace(',', '')
            self.data[col] = self.data[col].apply(lambda x: float(x))
            
        # Column values needs to be cleaned
        # FranchiseCode -> IsFranchise 1, 0
        self.data.loc[(self.data['FranchiseCode'] <= 1), 'IsFranchise'] = 0
        self.data.loc[(self.data['FranchiseCode'] > 1), 'IsFranchise'] = 1
        self.data['IsFranchise'] = self.data.IsFranchise.astype(int)

        # NewExist (drop NewExist=0) -> NewBusiness 1, 0
        self.data = self.data[(self.data['NewExist'] == 1) | (self.data['NewExist'] == 2)]
        self.data.loc[(self.data['NewExist'] == 1), 'NewBusiness'] = 0
        self.data.loc[(self.data['NewExist'] == 2), 'NewBusiness'] = 1

        # RevLineCr 
        self.data['RevLineCr'].loc[(self.data['RevLineCr'] == 'N') | (self.data['RevLineCr'] == 0)] = 0
        self.data['RevLineCr'].loc[(self.data['RevLineCr'] == 'Y') | (self.data['RevLineCr'] == 1)] = 1
        self.data['RevLineCr'].loc[(self.data['RevLineCr'] != 'Y') & (self.data['RevLineCr'] != 1) & (self.data['RevLineCr'] != 'N') & (self.data['RevLineCr'] != 0)] = 2

        # LowDoc
        self.data['LowDoc'].loc[(self.data['LowDoc'] == 'N') | (self.data['LowDoc'] == 0)] = 0
        self.data['LowDoc'].loc[(self.data['LowDoc'] == 'Y') | (self.data['LowDoc'] == 1)] = 1
        self.data['LowDoc'].loc[(self.data['LowDoc'] != 'Y') & (self.data['LowDoc'] != 1) & (self.data['LowDoc'] != 'N') & (self.data['LowDoc'] != 0)] = 2


        #NAICS -> Industry
        self.data['Industry'] = self.data['NAICS'].astype('str').apply(lambda x: x[:2])
        # Mapping the industry code with the industry code with the industry name
        self.data['Industry'] = self.data['Industry'].map({
            '11': 'Ag/For/Fish/Hunt',
            '21': 'Min/Quar/Oil_Gas_ext',
            '22': 'Utilities',
            '23': 'Construction',
            '31': 'Manufacturing',
            '32': 'Manufacturing',
            '33': 'Manufacturing',
            '42': 'Wholesale_trade',
            '44': 'Retail_trade',
            '45': 'Retail_trade',
            '48': 'Trans/Ware',
            '49': 'Trans/Ware',
            '51': 'Information',
            '52': 'Finance/Insurance',
            '53': 'RE/Rental/Lease',
            '54': 'Prof/Science/Tech',
            '55': 'Mgmt_comp',
            '56': 'Admin_sup/Waste_Mgmt_Rem',
            '61': 'Educational',
            '62': 'Healthcare/Social_assist',
            '71': 'Arts/Entertain/Rec',
            '72': 'Accom/Food_serv',
            '81': 'Other_no_pub',
            '92': 'Public_Admin'
        })
        self.data['Industry'] = self.data['Industry'].fillna('Undefined')
        
        # ZeroBal: If the balance was zero or not (If zero = 1, non-zero = 0) 
        self.data['ZeroBal'] = self.data['BalanceGross'].apply(lambda x: 1 if x == 0 else 0)
        
        # SBAGrAppv: If SBA_Appv = GrAppv 
        self.data['SBAGrAppv'] = 0
        self.data['SBAGrAppv'].loc[self.data['SBA_Appv'] == self.data['GrAppv']] = 1
        self.data['SBAGrAppv'].loc[self.data['SBA_Appv'] != self.data['GrAppv']] = 0
        
        # EntScale : Small enterprises: 10 to 49 employees. Medium-sized enterprises: 50 to 249 employees. Large enterprises: 250 employees or more.
        self.data['EntScale'] = 0
        self.data['EntScale'].loc[self.data['NoEmp'] < 50] = 'Small'
        self.data['EntScale'].loc[(self.data['NoEmp'] >= 50) & (self.data['NoEmp'] < 250)] = 'Medium'
        self.data['EntScale'].loc[self.data['NoEmp'] >= 250] = 'Large'

        # GDPTopState : If borrower’s State is in TOP 10 highest GDP states
        self.data['GDPTopState'] = self.data['State'].apply(lambda x: 1 if x in ['NY', 'MA', 'WV', 'CA', 'ND', 'CT', 'DE', 'AK', 'NE', 'IL'] else 0)
        
        # SameState : BankState = State then 1, else 0
        self.data['SameState']=0
        self.data['SameState'].loc[self.data['BankState'] == self.data['State']] = 1
        self.data['SameState'].loc[self.data['BankState'] != self.data['State']] = 0

        # LargeSBA : SBAAppv is greater than $200,000
        self.data['LargeSBA'] = 0
        self.data['LargeSBA'].loc[self.data['SBA_Appv'] > 200000] = 1
        
        # EmpGrowth : The value of CreateJob - RetainedJob.
        self.data['EmpGrowth'] = self.data['CreateJob'] - self.data['RetainedJob']
        self.data['EmpGrowth'].loc[self.data['EmpGrowth'] < 0] = 0
        
        # LoanAmountRatio : Divide the DisbursementGross column by the SBA_Appv column
        self.data['LoanAmountRatio'] = self.data['DisbursementGross'] / self.data['SBA_Appv']
        
        # BigBank : If the bank is in US top 15 bank list
        top_bank = [ 'JPMorgan Chase', 
                    'Bank of America', 
                    'Citigroup', 
                    'Wells Fargo', 
                    'U.S. Bancorp', 
                    'PNC Financial Services', 
                    'Truist Financial', 
                    'Goldman Sachs', 
                    'Capital One Financial', 
                    'TD Group US Holdings', 
                    'Bank of New York Mellon', 
                    'State Street', 
                    'Citizens Financial', 
                    'First Republic Bank', 
                    'Morgan Stanley Private Bank']

                    
        top_bank_initial = []
        for name in top_bank:
            top_bank_initial.append(name[0:11])
        top_bank_initial
        top_bank_initial = list(map(str.lower, top_bank_initial))
        self.data['Bank_init'] = self.data['Bank'].str.lower().str[:11]
        self.data['BigBank'] = self.data['Bank_init'].apply(lambda x: 1 if x in top_bank_initial else 0)
        
        # Drop columns that we don't need anymore
        col_to_drop = ['NAICS', 'NewExist', 'FranchiseCode', 'Bank_init']
        self.data_new = self.data.drop(columns = col_to_drop)
        self.data_new = self.data_new.reset_index(drop=True)
        
        # Define Categorical Variables & Numerical Variables
        self.categorical_var = ['City', 'State', 'Zip', 'Bank', 'BankState', 'UrbanRural', 
                   'RevLineCr', 'LowDoc', 'LoanInd', 'IsFranchise', 'Industry', 
                   'ZeroBal', 'SBAGrAppv', 'EntScale', 'GDPTopState', 
                   'SameState', 'LargeSBA', 'BigBank']
        self.numerical_var = []
        for col in self.data_new.columns:
            if col not in self.categorical_var + ['MIS_Status']:
                self.numerical_var.append(col)
        self.numerical_var

        self.data_new[self.categorical_var] = self.data_new[self.categorical_var].astype(object)


In [10]:
def project_1_scoring(data):
    
    # Cleanse dataset with Data_loader
    data_load=Data_loader(data)

    X = data_load.data_new
    # Data cleaning
    
    #Load Artifacts
    with open('artifact_dict.pkl','rb')as f:
        artifacts_dict=pickle.load(f)
    

    model=h2o.load_model('GBM_model_python_1683424268951_7550')
    
    
    cat_encoders = artifacts_dict["cat_encoders"]
    ohe_columns = artifacts_dict["ohe_columns"]

    scalers= artifacts_dict['num_scalers']
    scale_cols=artifacts_dict['continuous_columns']

    te_encoders=artifacts_dict['te_encoders']
    te_columns=artifacts_dict["te_columns"]
    
    columns_to_score = artifacts_dict["columns_to_train"]
    threshold = artifacts_dict["threshold"]
    
    for col in scale_cols :
        X[col+"_sc"] = scalers[col][0].transform(X[[col]])
    
    '''Encode categorical columns'''
    for col in ohe_columns:
        enc = cat_encoders[col][0]
        result = enc.transform(X[[col]])
        ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]]
        result_train = pd.DataFrame(result, columns=ohe_columns)
        X = pd.concat([X, result_train], axis=1)
    
    X_target_enc=te_encoders.transform(X[te_columns])
    
    for col in te_columns:
        X[col+'_trg'] = X_target_enc[col]
        
    X=H2OFrame(X)
        
    y_prob=model.predict(X)
    
    y_pred=y_prob['p1'].as_data_frame()['p1'].apply(lambda x :round(x)).tolist()
    

    d = {#'index':X.as_data_frame().index,
         "label":y_pred,
         "probability_0":y_prob['p0'].as_data_frame()['p0'],
         "probability_1":y_prob['p1'].as_data_frame()['p1']}
    
    return pd.DataFrame(d)

In [13]:
data_path = ""
project_1_scoring(data_path)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


,label,probability_0,probability_1
0,0,0.955585,0.044415
1,0,0.932871,0.067129
2,0,0.911221,0.088779
3,0,0.911426,0.088574
4,0,0.858613,0.141387
...,...,...,...
806394,0,0.928066,0.071934
806395,0,0.545852,0.454148
806396,0,0.725214,0.274786
806397,0,0.828325,0.171675
